In [2]:
import numpy as np
import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
from data_loaders import *
from missing_process.block_rules import *
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score


In [3]:


real_datalist = [
    "banknote",
        "concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]

missingtypelist = [
                    "quantile",
                   "diffuse",
                   "logistic"
                   ]

seed = 1
nfold = 5

In [4]:
def load_impute_data(missingtype,model_name,rule_name,dataname,fold,seed = 1):

    train_impute = np.load(f'impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_train.npy')
    test_impute = np.load(f'impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_test.npy')
    return train_impute,test_impute

def load_train_test(index_file,norm_values,observed_masks,label_values):  

    train_index = index_file["train_index"]
    test_index = index_file["test_index"]

    train_values = norm_values[train_index,:]

    train_masks = observed_masks[train_index,:]

    test_values = norm_values[test_index,:]

    test_masks = observed_masks[test_index,:]


    train_label = label_values[train_index]

    test_label = label_values[test_index]

    return train_values,train_masks,train_label,test_values,test_masks,test_label


def process_target(dataname,y):
    if dataname in ["concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","yacht_hydrodynamics"
            ]:
        return y,"reg"
    
    else:
        encoder = LabelEncoder()
        y_encoded = encoder.fit_transform(y.reshape(-1, 1))
        return y_encoded,"clf"
    

In [10]:
def fillin_imputed_data(imputed,mask,original):
    filled_data = np.where(mask == 1, original, imputed)
    return filled_data

In [11]:
fillin_imputed_data(np.array([1,2,3]),np.array([1,0,0]),np.array([4,5,6]))

array([4, 2, 3])

In [16]:
# # Single Model Name

# datalist = real_datalist
# model_name = "tabcsdi"
# missingtypelist = ["logistic"]
# for missingtype in missingtypelist:
#     if missingtype == "logistic":
#         missing_rule = load_json_file("missing_rate.json")
#     elif missingtype == "diffuse":
#         missing_rule = load_json_file("diffuse_ratio.json")
#     elif missingtype == "quantile":
#         missing_rule = load_json_file("quantile_full.json")

#     d_v = {}
#     d_AMI = {}

#     # load data and its mask
#     for dataname in tqdm(datalist):
        
#         directory_path = f"datasets/{dataname}"
#         data = dataset_loader(dataname)    
#         norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')

#         # clustering = DBSCAN().fit(norm_values)
#         # #print(clustering.labels_)
#         # ncluster = len(set(clustering.labels_))
#         # if ncluster == 1:
#         #     print(dataname)
#         d_v[dataname] = []
#         d_AMI[dataname] = []

#         for rule_name in tqdm(missing_rule):
            
#             observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')
#             f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
#             index_file = json.load(f)

#             v_list = []
#             AMI_list = []
#             for fold in index_file:
#                 index = index_file[fold]
#                 train_values,train_masks,train_label,test_values,test_masks,test_label = load_train_test(index,norm_values,observed_masks)
#                 impute_train,impute_test  = load_impute_data(missingtype,model_name,rule_name,dataname,fold)

#                 impute_test = fillin_imputed_data(impute_test,test_masks,test_values)
#                 # print(impute_test)
#                 # print(test_values)
#                 # print(test_masks)
#                 clu_test = DBSCAN().fit(test_values).labels_
#                 clu_test_imp = DBSCAN().fit(impute_test).labels_

                
#                 V = v_measure_score(clu_test, clu_test_imp)
#                 AMI = adjusted_mutual_info_score(clu_test, clu_test_imp)

#                 v_list.append(V)
#                 AMI_list.append(AMI)
            

#             d_v[dataname].append(np.mean(v_list))
#             d_AMI[dataname] .append(np.mean(AMI_list))

#     df_v = pd.DataFrame(d_v,index=[rule_name for rule_name in missing_rule])
#     df_AMI = pd.DataFrame(d_AMI,index=[rule_name for rule_name in missing_rule])

#     path = f"clustering/{missingtype}/"
#     if not os.path.exists(path):
#         os.makedirs(path)

#     df_v.to_csv(f"{path}/{model_name}_V.csv")

#     df_AMI.to_csv(f'{path}/{model_name}_AMI.csv')



In [6]:
# def fillin_imputed_data(imputed, mask, original):

#     filled_data = np.where(np.isnan(mask), 1, imputed)
#     return filled_data

In [12]:

datalist = real_datalist
model_name_list = ["random","zero","Mean","MF","KNN","MICE","Missforest","XGB","OT","Hyper","GAIN","notmiwae","miwae","tabcsdi"]

missingtypelist = ["logistic"]
rule_name = 0.5

real_datalist = [
    "banknote",
        "concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]

reg_data = ["concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","yacht_hydrodynamics"
            ]
clf_data =  ["banknote","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast",
            ]

def run_ml(rule_name,model_number,missingtype,datalist,task_type ="reg"):

    d_f1_rmse = {}
    d_acc_r2= {}


    for dataname in tqdm(datalist):
        
        directory_path = f"datasets/{dataname}"
        data = dataset_loader(dataname)    
        norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')
        label_values, task_type = process_target(dataname,data["target"])

        if task_type =="reg":
           ml_model_list = [Ridge(), MLPRegressor(random_state=1), SVR()]
           
        else:
            ml_model_list = [LogisticRegression(random_state=1), MLPClassifier(random_state=1), SVC()]

        model = ml_model_list[model_number]

        observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')
        f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
        index_file = json.load(f)

        d_f1_rmse[dataname] = {}
        d_acc_r2[dataname] = {}

        for model_name in model_name_list:
            d_f1_rmse[dataname][model_name]=0
            d_acc_r2[dataname][model_name]=0
            # load data and its mask

            f1_rmse_list = []
            acc_r2_list = []
            for fold in index_file:
                index = index_file[fold]
                train_values,train_masks,train_label,test_values,test_masks,test_label = load_train_test(index,norm_values,observed_masks,label_values)
                impute_train,impute_test  = load_impute_data(missingtype,model_name,rule_name,dataname,fold)

                impute_test = fillin_imputed_data(impute_test,test_masks,test_values)
                impute_test = np.nan_to_num(impute_test, nan=0)

                try:
                    evl1,evl2 = model_eval(train_label, impute_train, impute_test, test_label, task_type,model)

                except:
                    print(model_name)
                    evl1,evl2 = np.nan,np.nan

                f1_rmse_list.append(evl1)
                acc_r2_list.append(evl2)
            

            d_f1_rmse[dataname][model_name]=np.mean(f1_rmse_list)
            d_acc_r2[dataname][model_name]=np.mean(acc_r2_list)


    d_f1_rmse = pd.DataFrame(d_f1_rmse).T
    d_acc_r2 = pd.DataFrame(d_acc_r2).T


    path = f"ml_new/{missingtype}/"
    if not os.path.exists(path):
        os.makedirs(path)

    
    if task_type == "reg":

        d_f1_rmse.to_csv(f"{path}/{rule_name}_{task_type}_{model_number}_rmse.csv")

        d_acc_r2.to_csv(f'{path}/{rule_name}_{task_type}_{model_number}_r2.csv')
    else:
        d_f1_rmse.to_csv(f"{path}/{rule_name}_{task_type}_{model_number}_f1.csv")

        d_acc_r2.to_csv(f'{path}/{rule_name}_{task_type}_{model_number}_acc.csv')


In [14]:
missingtype = "diffuse"
missingtype = "logistic"
model_name_list = ["random","zero","Mean","MF","KNN","MICE","Missforest","XGB","OT","Hyper","GAIN","notmiwae","miwae","tabcsdi"]
reg_data = ["concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","yacht_hydrodynamics"
            ]
clf_data =  ["banknote","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast",
            ]


for missingtype in ["diffuse","logistic"]:
    for rule_name in ["0.3","0.5","0.7"]:
        for model_number in [0,1,2]:
            run_ml(rule_name,model_number,missingtype,reg_data,task_type ="reg")
            run_ml(rule_name,model_number,missingtype,clf_data,task_type ="clf")

 20%|██        | 1/5 [00:00<00:00,  8.55it/s]d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=8.89288e-12): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 60%|██████    | 3/5 [00:00<00:00,  8.00it/s]

XGB


  0%|          | 0/5 [00:00<?, ?it/s]d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
 20%|██        | 1/5 [00:00<00:02,  1.47it/s]d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
 40%|████      | 2/5 [00:01<00:02,  1.14it/s]d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\linear_mode

In [ ]:
missingtype = "quantile"
for rule_name in ["Q1_Q4_0.5","Q2_Q3_0.5","Q2_Q4_0.5"]:
    for model_number in [0,1,2]:
        #run_ml(rule_name,model_number,missingtype,reg_data,task_type ="reg")
        run_ml(rule_name,model_number,missingtype,clf_data,task_type ="clf")

 40%|████      | 2/5 [00:00<00:00,  7.23it/s]d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 60%|██████    | 3/5 [00:00<00:00,  8.10it/s]

XGB
XGB
XGB


  0%|          | 0/5 [00:00<?, ?it/s]d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\miniconda3\envs\py3.10\lib\s

XGB


  0%|          | 0/5 [00:00<?, ?it/s]d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\miniconda3\envs\py3.10\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\miniconda3\envs\py3.10\lib\s

In [ ]:
missingtype = "quantile"
for rule_name in ["Q1_Q4_0.5","Q2_Q3_0.5","Q2_Q4_0.5"]:
    for model_number in [0,1,2]:
        run_ml(rule_name,model_number,missingtype,reg_data,task_type ="reg")

In [13]:
import numpy as np
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, r2_score

def model_eval(label_train, impute_train, impute_test, label_test, task_type,model):
    #print(task_type)
    if task_type == "reg":
        # Define regressors
        reg = model
        
        # Check for NaNs

        reg.fit(impute_train, label_train)
        y_pred_test = reg.predict(impute_test)



        test_rmse = np.sqrt(mean_squared_error(label_test, y_pred_test))
        r2 = r2_score(label_test, y_pred_test)
        return test_rmse,r2

    else:
        # Define classifiers

        clf = model
        # Evaluate each classifier and store F1 scores for both train and test

        clf.fit(impute_train, label_train)
        y_pred_test = clf.predict(impute_test)
   
        # Calculate average F1 score
        test_f1 = f1_score(label_test, y_pred_test, average='macro')
        accuracy = accuracy_score(label_test, y_pred_test, )
        return test_f1, accuracy
